# <center>Network Science</center>

## <center>Home Assignment #3: Centralities and Assortativity</center>

### <center>Student: *{Рубцов Василий}*</center>

#### <hr /> General Information

**Due Date:** 13.03.2016 23:59 <br \>
**Late submission policy:** -0.2 points per day <br \>


Please send your reports to <network.hse.2016@gmail.com> with message subject of the following structure:<br \> **[HSE Networks 2015] *{LastName}* *{First Name}* HA*{Number}***

Support your computations with figures and comments. <br \>
If you are using IPython Notebook you may use this file as a starting point of your report.<br \>
<br \>
<hr \>

## Problems

### Task 1

Compute degree centrality, Pagerank and  HubAuthorities scores for the [flickr](https://www.hse.ru/data/2016/02/24/1139618969/flickr.mat) network. 

Data contains sparse matrix A and list of user names.
This is a “denser” part of the Flickr photo sharing site friendship graph from 2006. Edge direction corresponds to friendship requests (following). Some of the links are reciprocal,others not.  

Provide top 50 names in each ranking, compare results

You can load .mat files with the following commands:

In [45]:
import scipy.io
import networkx as nx
import numpy as np
data = scipy.io.loadmat('flickr.mat')
import matplotlib.pyplot as plt
%matplotlib inline
import re
from astropy.table import Table, Column

In [51]:
names = [re.sub("\s\s+", '', st) for st in data['names']]
A = data['A']

strind = scipy.sparse.find(A)[0]
colind = scipy.sparse.find(A)[1]

# create a directed graph
G = nx.DiGraph()
G.add_nodes_from(names)

for i in range(0, len(strind)):
    G.add_edge(names[strind[i]],names[colind[i]])

    
#let's compute in and out degree centrality

dc_in = nx.in_degree_centrality(G)
degtop_in = sorted(dc_in, key=dc_in.__getitem__, reverse = True)[:50] #50 users with the higher degree centrality

dc_out = nx.out_degree_centrality(G)
degtop_out = sorted(dc_out, key=dc_out.__getitem__, reverse = True)[:50]

In [47]:
#Let's compute pagerank 
pr = nx.pagerank(G)
prtop = sorted(pr, key=pr.__getitem__, reverse = True)[:50]

In [48]:
#let's compute hubs and authorities and the 50th best
h,a = nx.hits(G)

hh = sorted(h, key = h.__getitem__, reverse = True)[:50]

In [49]:
aa = sorted(a, key = a.__getitem__, reverse = True)[:50]

In [52]:
t = Table([degtop_in, degtop_out, prtop, hh, aa], names=('in degree_centrality','out degree_centrality','page_rank','hubs', 'authorities'))
t

in degree_centrality,out degree_centrality,page_rank,hubs,authorities
str576,str512,str576,str576,str576
awfulsara,anildash,awfulsara,mrpiink,awfulsara
drp,tozzer,drp,automat,drp
*Ivan*,AtiRanA,antimethod,schizoo23,DrJoanne
antimethod,pixietart,BombDog,lorrainemd,*Ivan*
DrJoanne,jakedobkin,*Ivan*,sgoralnick,antimethod
BombDog,Buntekuh,MaD GiÂ®Lâ¢â¢,*starlet*,BombDog
Simon Pais,brainware3000,DrJoanne,brynfoto,Simon Pais
deborah lattimore,Jakes_World,Simon Pais,liquid pixel,deborah lattimore
MaD GiÂ®Lâ¢â¢,maximolly,deborah lattimore,noahstone,cymagen


### <hr /> Task 2

Here are the [Facebook friendship graphs](https://www.hse.ru/data/2016/03/05/1125656999/universities.zip) from several US universities from 2005 (one year after fb launch).

Data contains a A matrix (sparse) and a "local_info" variable, one row per node: 
a student/faculty status flag, gender, major, second major/minor (if applicable), dorm/house, year, and high school. 
Missing data is coded 0.

Compute node degree assortativity (mixining by node degree) and assortativity coefficient (modularity) for gender, major, dormitory, year, high school for all universities and compare the results.

In [41]:
#Let's define the function which will be returns the coefficient of assortative mixing of each categories
def assort(string):
    dataB = scipy.io.loadmat(string)
    GB = nx.from_scipy_sparse_matrix(dataB['A'], create_using=nx.DiGraph())
    loc_infB = dataB['local_info']
    l = [nx.degree_assortativity_coefficient(GB)]
    for i in range(7):
        nx.set_node_attributes(GB, i ,{k:loc_infB[k][i] for k in range(len(loc_infB))})
        l.append(nx.attribute_assortativity_coefficient(GB, i))
    return l

In [42]:
#let's print assortative mixing in each univesity
uni = ['degree assortativity', 'status flag', 'gender', 'major', 'minor', 'living', 'year','high school']
tab = Table([uni, assort('Berkeley13.mat'), assort('Caltech36.mat'), assort('Harvard1.mat'), assort('Oklahoma97.mat'), assort('Princeton12.mat'), assort('Texas80.mat') ], 
            names=('University', 'Berkeley','Caltech','Harvard', 'Oklahoma', 'Princeton', 'Texas'))
nn = ['Berkeley','Caltech','Harvard', 'Oklahoma', 'Princeton', 'Texas']
for n in nn:
    tab[n].format = '%.4f'
tab

University,Berkeley,Caltech,Harvard,Oklahoma,Princeton,Texas
str640,float64,float64,float64,float64,float64,float64
degree assortativity,0.0103,-0.0653,0.1451,0.0737,0.0911,0.1639
status flag,0.2746,0.2441,0.5426,0.1946,0.4822,0.2886
gender,0.0409,0.0463,0.0453,0.0928,0.0506,0.0797
major,0.0583,0.0347,0.0497,0.0364,0.0544,0.0597
minor,0.0166,0.0107,0.0252,0.0120,0.0152,0.0188
living,0.1473,0.3492,0.1257,0.2803,0.0947,0.2048
year,0.3081,0.2375,0.4212,0.2012,0.4467,0.2462
high school,0.0774,0.0027,0.0240,0.1395,0.0188,0.1480
